## 월간공정 현황 수집

In [1]:
from common import commonFunc as cf
import pandas as pd
import threading


pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "월간공정 현황"
APIKEY = "11067238-08C1-4FDE-857B-4C865C85F2A7"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/월간공정 현황/월간공정 현황.csv'

In [2]:
g_totalCount = 0

In [3]:
#임시 
imsiDf = pd.read_csv("../output/건설사업정보시스템/연도별 공사계약 목록/연도별 공사계약 목록.csv", encoding="ms949")

In [4]:
dfCombi = imsiDf.loc[:,["sptno","sptto"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]
dfCombiLen

7268

In [5]:
numOfThread = 10

resultDfMerged = list(pd.DataFrame() for x in range(numOfThread))


In [6]:
##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
SERVICENAME

'selectMlyPrpoList'

In [7]:
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]
PRIMARYKEY

'serviceKey,sptNo,stpTo,rprtYm'

In [8]:
JSONKEY = "detail1"
DUMMY = 0

In [9]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1
PAGEYN

0

## 보고년월 기간을 추출하기 위한 커스텀 함수

In [10]:
from datetime import datetime

# d1, d2 : datetime format 
# d1 >= d2
def calc_month(d1, d2):
    return (d1.year - d2.year) * 12 + (d1.month - d2.month + 1)

In [11]:
# #### 날짜 생성 ####

# import datetime

# startDateTime = 199701
# now = datetime.datetime.now()
# currentDateTime = now.strftime("%Y%m")
# dateRange = list(map(int,pd.period_range(199701,currentDateTime,freq='M').strftime('%Y%m').tolist()))

## 월간공정 현황 전용 스크랩 함수

In [12]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os
import time

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            time.sleep(1)
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

            response = requests.get(inUrl+queryParams, verify=False)
            print(inUrl+queryParams)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("{} page is empty".format(i))
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 detailList1~5까지 봐야 한다.
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)
                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    if(response.json()["response"]["body"].get('detail1') != None):
                        print("we have a data on detail1")
                        jsondataDetail = response.json()["response"]["body"]["detail1"]

                    elif(response.json()["response"]["body"].get('detail2') != None):
                        jsondataDetail = response.json()["response"]["body"]["detail2"]
                        print("we have a data on detail2")

                    elif(response.json()["response"]["body"].get('detail3') != None):
                        jsondataDetail = response.json()["response"]["body"]["detail3"]
                        print("we have a data on detail3")

                    elif(response.json()["response"]["body"].get('detail4') != None):
                        jsondataDetail = response.json()["response"]["body"]["detail4"]
                        print("we have a data on detail4")

                    elif(response.json()["response"]["body"].get('detail5') != None):
                        jsondataDetail = response.json()["response"]["body"]["detail5"]
                        print("we have a data on detail5")
#                     print(jsondataDetail)
                    rowDataDetail = pd.DataFrame(jsondataDetail)
                    emptyPd = pd.merge(emptyPd, rowDataDetail, on=['sptNo','sptTo'])
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [13]:
lock = threading.Lock()
def threadReq(startIdx, endIdx,x):
    resultDf = pd.DataFrame()
    global resultDfMerged
    dateRange = pd.period_range(resultDf["stwrdt"].values[0],resultDf["ccwdt"].values[0],freq='M').strftime('%Y%m').tolist()
            
    for i in range(startIdx,endIdx):
        sptnoValue = dfCombi.loc[i].sptno
        spttoValue = dfCombi.loc[i].sptto
        BASEPARAM={"serviceKey":APIKEY,"sptNo": sptnoValue,"sptTo":spttoValue,"rprtYm": "", "type":"json"}
        resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
        print("i= ",i)
        if resultDf.empty: # 정상 데이터가 없는 경우
            print("No Data")
        else:
            for i in range(0,len(dateRange)):
                BASEPARAM={"serviceKey":APIKEY,"sptNo": sptnoValue,"sptTo":spttoValue,"rprtYm": str(dateRange[i]), "type":"json"}
                resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
                lock.acquire()
                resultDfMerged[x] = resultDfMerged[x].append(resultDf) 
                lock.release()


In [15]:

resultDfLastMerged = pd.DataFrame()
section = list(range(0, dfCombiLen, int(dfCombiLen/numOfThread))) # we have "numOfThread +1" sections

# dfCombiLen = 6 # for test
if dfCombiLen < numOfThread:
    print("데이터의 갯수가 적어서 쓰레드를 시작할 수 없습니다.")
else:
    x_ls =list(range(numOfThread))
    thread_list = []
    results = []

    for x in x_ls:
        startIdx = section[x]
        endIdx =  section[x+1] -1 if (x+1 <= numOfThread -1) else dfCombiLen - 1
        print("start ",startIdx, " end ",endIdx)
        mythread = threading.Thread(target=threadReq, args=(startIdx, endIdx+1,x))
        thread_list.append(mythread)
    for i in thread_list:
        i.start()
    for i in thread_list:
        i.join()
    for i in range(0,numOfThread):
        resultDfLastMerged = resultDfLastMerged.append(resultDfMerged[i]) 



start  0  end  725
start  726  end  1451
start  1452  end  2177
start  2178  end  2903
start  2904  end  3629
start  3630  end  4355
start  4356  end  5081
start  5082  end  5807
start  5808  end  6533
start  6534  end  7267


Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3361, in get_loc
Exception in thread Thread-9Exception in thread :
Thread-10:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3361, in get_loc
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3361, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 76, in pandas._libs.index.IndexEngine.get_loc
Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexes\base.py", line 3361, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 76, in pandas._libs.index.IndexEngine.get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index

In [16]:
cf.savedata(resultDfLastMerged, SITENAME,DATANAME,SERVICENAME)

월간공정 현황 save compled


In [17]:
resultDfLastMerged.shape

(0, 0)